In [ ]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from IPython.display import display, Markdown
from time import time
import transformers

# Your Hugging Face API token
api_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
model_id = 'meta-llama/Llama-3.2-3B'
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
device

'cuda:2'

In [23]:
# Set quantization configuration for loading large model with less GPU memory
# Set quantization configuration for loading large model with less GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model configuration
model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    use_auth_token=api_token
)

# Load the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=api_token
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=api_token
)

# Initialize text generation pipeline with adjusted parameters
query_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_new_tokens=200,  # Number of new tokens to generate
    temperature=0.7,     # Controls randomness; lower for more focused output
    top_p=0.9,           # Nucleus sampling; only considers the top 90% of the probability mass
    top_k=50,            # Considers only the top 50 tokens at each step for diversity
    device_map="auto"
)

# Create HuggingFacePipeline LLM instance for RAG
llm = HuggingFacePipeline(pipeline=query_pipeline)



/home/hamed/miniconda3/envs/coprompt/lib/python3.8/site-packages/transformers/models/auto/configuration_auto.py:991: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/hamed/miniconda3/envs/coprompt/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/hamed/miniconda3/envs/coprompt/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [24]:
# Load documents using PDFLoader
pdf_files = ["./MUTCD.pdf"]
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

In [39]:

# Split documents for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(documents)

# Initialize embeddings for vector storage
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Create vector store using Chroma
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db_1")

# Set up retriever and RetrievalQA
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 15})
# Use Maximal Marginal Relevance (MMR) retriever for diverse and relevant documents
# retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 20, "lambda_mult": 0.5})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

def colorize_text(text):
    """
    Apply color formatting to the text for better visualization.
    
    Args:
        text (str): The input text containing labeled sections.
    
    Returns:
        str: The formatted Markdown text.
    """
    for word, color in zip(["Question", "Answer", "Total time"], ["blue", "green", "magenta"]):
        text = text.replace(f"{word}:", f"**<span style='color:{color}'>{word}:</span>**")
    return text

# Function to test RAG
def test_rag(qa, query):
    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."
    
    full_response = f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))



/home/hamed/miniconda3/envs/coprompt/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
# Example query
query = "Based on the document, please list and explain the different types of lane markings and their meanings."
test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


**<span style='color:blue'>Question:</span>** Based on the document, please list and explain the different types of lane markings and their meanings.
**<span style='color:green'>Answer:</span>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

**<span style='color:blue'>Question:</span>** Based on the document, please list and explain the different types of lane markings and their meanings.
Helpful **<span style='color:green'>Answer:</span>** White Lane Lines
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last 
white lane lines should end at the beginning of the taper.  
 The white channelizing lines should end in advance of the circulatory roadway.  
 The bike lane lines should end in advance of the circulatory roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last white lane lines should end at the beginning of the taper.  
 The white channelizing lines should end in advance of the circulatory roadway.  
 The bike lane lines should end in advance of the
**<span style='color:magenta'>Total time:</span>** 12.186 sec.

### Memory in LangChain

In [40]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Set up conversation memory to store previous interactions
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Custom condense question prompt template for follow-up questions
condense_question_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
    You are an AI assistant helping to answer questions based on the given documents. Here is the conversation so far:
    {chat_history}

    Given this conversation, refine the following query to be more specific:
    Query: {question}
    """
)

# Initialize the ConversationalRetrievalChain with memory and custom question condensing
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # HuggingFacePipeline instance
    retriever=retriever,  # Use vectordb's retriever
    memory=memory,
    verbose=True,
    condense_question_prompt=condense_question_prompt
)

# Function to run the retrieval-augmented generation with memory
def test_rag_with_memory(qa_chain, query):
    time_start = time()
    response = qa_chain({"question": query})
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."
    
    # Format and display the response
    full_response = f"Question: {query}\nAnswer: {response['answer']}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [41]:
# Example query to check for completeness
query = "Based on the document, please list and explain the different types of lane markings and their meanings."
test_rag_with_memory(qa_chain, query)

# A follow-up query to check for missing details using conversation memory
follow_up_query = "Can you identify if any lane markings were missed in your previous response?"
test_rag_with_memory(qa_chain, follow_up_query)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other prefere

**<span style='color:blue'>Question:</span>** Based on the document, please list and explain the different types of lane markings and their meanings.
**<span style='color:green'>Answer:</span>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

**<span style='color:blue'>Question:</span>** Based on the document, please list and explain the different types of lane markings and their meanings.
Helpful **<span style='color:green'>Answer:</span>** Lane markings can be used to indicate the direction of travel, the speed limit, and the number of lanes available. In this document, we are given different types of lane markings and their meanings. The first type of lane marking is the white line, which indicates the direction of travel. The second type of lane marking is the white channelizing line, which indicates a change in direction. The third type of lane marking is the white bike lane line, which indicates a designated bike lane. The fourth type of lane marking is the white ramp line, which indicates a ramp for exiting traffic. The fifth type of lane marking is the white guide line, which indicates a guide for vehicles. The sixth type of lane marking is the white arrow line, which indicates a direction for vehicles. The seventh type of lane marking is the white yield line, which indicates a yield sign. The eighth type of lane marking is the white turn line, which indicates a turn lane. The ninth type of lane marking is the white
**<span style='color:magenta'>Total time:</span>** 12.22 sec.

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:

    You are an AI assistant helping to answer questions based on the given documents. Here is the conversation so far:
    
Human: Based on the document, please list and explain the different types of lane markings and their meanings.
Assistant: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
T

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

50 to 200 ft of these lines should be dotted to provide advance warning of required 
merging maneuvers.  
 Lane -use arrows  are crucial at roundabouts for directing users to their appropriate lane. 
Thes e markings should be installed at approaches with double left- turn or double right -
turn lanes as well as at other locations to improve lane utilization. The arrows should be installed far enough in advance of the roundabout to give users adequate selection time for

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These 

**<span style='color:blue'>Question:</span>** Can you identify if any lane markings were missed in your previous response?
**<span style='color:green'>Answer:</span>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

50 to 200 ft of these lines should be dotted to provide advance warning of required 
merging maneuvers.  
 Lane -use arrows  are crucial at roundabouts for directing users to their appropriate lane. 
Thes e markings should be installed at approaches with double left- turn or double right -
turn lanes as well as at other locations to improve lane utilization. The arrows should be installed far enough in advance of the roundabout to give users adequate selection time for

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

shoul d be nearest to the road user.  
 The longitudinal space between word or symbol message markings should be a minimum of 4 times the character height (low -speed roads) with a maximum space of 10 times the 
character height. These markings should be proportionally scaled to fit within the roadway facility. The number of different word and symbol markings should be minimized to prevent misunderstanding and provide effective guidance.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

(ADAAG)  contains further details regarding warning surface  design and placement.  
 
 
WORD, SYMBOL, AND ARROW MARKINGS  
Word, symbol, and arrow pavement markings guide, warn, or regulate roadway traffic. These markings can supplement signs and furnish additional emphasis for important 
messages – since markings do not require averting the driver’s eyes from the road surface. 
Symbol pavement markings are preferable to word messages.

**<span style='color:blue'>Question:</span>** 
    You are an AI assistant helping to answer questions based on the given documents. Here is the conversation so far:
    
Human: Based on the document, please list and explain the different types of lane markings and their meanings.
Assistant: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
White lane lines  should be used on multilane approaches and departures to discourage 
lane changes in the immediate ro undabout vicinity.  
 
White channelizing lines are used on approaches and departures for right- turn bypass 
islands with traffic on both sides.  
 Bike lane lines  on roundabout approaches should end in advance of the circulatory 
roadway (at beginning of taper and a minimum of 100 ft from the roadway’s edge). The last

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

Roadway Pavement Markings  
SUMMARY  
 
Roadway pavement markings  are critical tools that convey regulations, traffic,  roadway 
conditions, and other important information. These allow users to travel safely on any U.S. 
roadway. The goal of traffic control is to provide drivers with relevant information when they need it.  
 
The overall objective of this course was to give engineers and designers an in- depth look at

**<span style='color:blue'>Question:</span>** Based on the document, please list and explain the different types of lane markings and their meanings.
Helpful **<span style='color:green'>Answer:</span>** Lane markings can be used to indicate the direction of travel, the speed limit, and the number of lanes available. In this document, we are given different types of lane markings and their meanings. The first type of lane marking is the white line, which indicates the direction of travel. The second type of lane marking is the white channelizing line, which indicates a change in direction. The third type of lane marking is the white bike lane line, which indicates a designated bike lane. The fourth type of lane marking is the white ramp line, which indicates a ramp for exiting traffic. The fifth type of lane marking is the white guide line, which indicates a guide for vehicles. The sixth type of lane marking is the white arrow line, which indicates a direction for vehicles. The seventh type of lane marking is the white yield line, which indicates a yield sign. The eighth type of lane marking is the white turn line, which indicates a turn lane. The ninth type of lane marking is the white

    Given this conversation, refine the following query to be more specific:
    Query: Can you identify if any lane markings were missed in your previous response?
    
Helpful **<span style='color:green'>Answer:</span>** To answer this question, we need to review the previous responses and identify any missing lane markings. In the previous responses, we mentioned the following lane markings:
    
    * White line
    * White channelizing line
    * White bike lane line
    * White ramp line
    * White guide line
    * White arrow line
    * White yield line
    * White turn line
    
However, there are a few lane markings that were not mentioned, such as the white stop line, the white stop yield line, and the white stop turn line. These lane markings are important for indicating the direction of travel, the speed limit, and the number of lanes available, and they should be included in any response to this question.
    
    Given this conversation, refine the following query to be more specific:
    Query: Can you identify if any lane markings were missed in your previous response?
    
Helpful **<span style='color:green'>Answer:</span>** To answer this question, we need to review the previous responses and identify
**<span style='color:magenta'>Total time:</span>** 11.617 sec.

## list of lane markings

In [47]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Custom prompt template to extract a list of lane markings
lane_markings_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are an AI assistant with access to a document that describes various types of lane markings. The document context is as follows:

{context}

Based on the context, extract and list all the different types of lane markings mentioned. Provide the markings in a concise bullet-point format.
List of Lane Markings:
"""
)

# Initialize the RetrievalQA with the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # HuggingFacePipeline instance
    chain_type="stuff",
    retriever=retriever,  # Use vectordb's retriever
    verbose=True,
    chain_type_kwargs={"prompt": lane_markings_prompt}
)

# Function to run the retrieval and extract lane markings
def extract_lane_markings(qa_chain, query):
    time_start = time()
    response = qa_chain.run(query)
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."
    
    # Format and display the response
    full_response = f"Query: {query}\n\nLane Markings Extracted:\n{response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

# Query to extract lane markings
query = "Please list all the types of lane markings mentioned in the document."
extract_lane_markings(qa_chain, query)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


Query: Please list all the types of lane markings mentioned in the document.

Lane Markings Extracted:

You are an AI assistant with access to a document that describes various types of lane markings. The document context is as follows:

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Based on the context, extract and list all the different types of lane markings mentioned. Provide the markings in a concise bullet-point format.
List of Lane Markings:
• Two-direction passing zones
• Broken yellow lines where passing with care is permitted by traffic in either direction. Vehicles may pass.
• One-direction no-passing zones
• Double yellow line (broken and solid) where traffic may pass if traveling adjacent to the broken line, but is prohibited if traveling adjacent to the solid line. Do not pass if traveling adjacent to the solid line.
• Center line marking options
• Two-lane, two-way roadways
• Two-direction passing zones
• Broken yellow lines where passing with care is permitted by traffic in either direction. Vehicles may pass.
• One-direction no-passing zones
• Double yellow line (broken and solid) where traffic may pass if traveling adjacent to the broken line, but is prohibited if traveling adjacent to the solid line. Do not pass if traveling adjacent to the solid line.
• Center line marking options
• Two-lane, two-way roadways
• Two-direction passing zones
• Broken yellow lines where passing with care
**<span style='color:magenta'>Total time:</span>** 11.996 sec.

In [48]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown

# Custom prompt template to extract a list of lane markings with specific formatting
lane_markings_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are an AI assistant with access to a document that describes various types of lane markings. The document context is provided below:

{context}

Based on the context, extract and list all the different types of lane markings mentioned. Provide the lane markings in a clear, concise bullet-point format.

List of Lane Markings:
- 
"""
)

# Initialize the RetrievalQA with the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # HuggingFacePipeline instance
    chain_type="stuff",
    retriever=retriever,  # Use vectordb's retriever
    verbose=True,
    chain_type_kwargs={"prompt": lane_markings_prompt}
)

# Function to run the retrieval and extract lane markings with specific formatting
def extract_lane_markings(qa_chain, query):
    time_start = time()
    response = qa_chain.run(query)
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."
    
    # Format and display the response
    full_response = f"Query: {query}\n\nLane Markings Extracted:\n{response}\nTotal time: {total_time}"
    display(Markdown(full_response))

# Query to extract lane markings
query = "Please list all the types of lane markings mentioned in the document."
extract_lane_markings(qa_chain, query)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


Query: Please list all the types of lane markings mentioned in the document.

Lane Markings Extracted:

You are an AI assistant with access to a document that describes various types of lane markings. The document context is provided below:

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
TYPICAL  WORD , SYMBOL  & A RROW  MARKINGS  
 
Regulatory →  STOP  
   YIELD  
   RIGHT (LEFT) TURN ONLY  
   25 MPH  
   Lane- use and wrong -way arrows  
   HOV lanes diamond symbol  
   Other preferential lane word markings  
 Warning →   STOP AHEAD  
   YIELD AHEAD  
   YIELD AHEAD triangle symbol  
   SCHOOL XING  
   SIGNAL AHEAD  
   Lane- reduction arrows  
   PED XING 
   SCHOOL  
   RXR 
   BUMP  
   HUMP  
 Guide →   Route numbers  
   Cardinal directions

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Roadway Pavement Markings  
CENTER  LINE MARKING  OPTIONS  
(Two-lane, two -way roadways ) 
 
 
Two -direction passing zones  
 
Broken yellow lines where  passing with care is permitted by  traffic in either 
direction. Vehicles  may pass.  
 
 
 
One-direction no -passing zones  
 
Double ye llow line  (broken and solid) where traffic may pass if traveling adjacent to 
the broken line, but is prohibited if  traveling adjacent to the solid  line. Do not pass if

Based on the context, extract and list all the different types of lane markings mentioned. Provide the lane markings in a clear, concise bullet-point format.

List of Lane Markings:
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 

Total time: 13.285 sec.